In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, date
import numpy as np
import pytz

In [4]:
list_symbols = pd.read_excel('/Users/alexandre/Desktop/FIIs/FiisPY/carteira.xlsx')['Código'].values
list_symbols

array(['BBAS3', 'BAZA3', 'EQTL3', 'BTCI11', 'VGIR11', 'TGAR11', 'XPML11',
       'HGLG11', 'KNRI11', 'VGIA11', 'XPCA11', 'BODB11', 'VGHF11',
       'MXRF11', 'PATL11'], dtype=object)

In [ ]:
list_melhores = []
list_relacao_mm = []
list_precos = []
for ativo in list_symbols:
    soma_fundamentos = 0
    dividend_yield = 0
    dados = yf.Ticker(ativo+'.SA')
    hist = dados.history(period='12mo', actions=True)
    hist_precos = hist['Close']
    hist_dividendos = hist['Dividends']
    hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2022,10,1, tzinfo=pytz.utc):datetime(2023,10,30, tzinfo=pytz.utc)]
    dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos.iloc[-1] if hist_precos.iloc[-1] > 0 else 0
    if dividend_yield > 0.03:
        try:
            balanco_anual = dados.balance_sheet
            balanco_trimestral = dados.quarterly_balance_sheet
            dre_anual = dados.financials
            dfc_anual = dados.cashflow
            dre_trimestral = dados.quarterly_financials
            dfc_trimestral = dados.quarterly_cash_flow
            if dre_anual.keys()[0].year >= 0:
                balanco = balanco_anual    
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'].values[0] > dre.loc['Total Revenue'].values[1])
                variacao_ebit = int(dre.loc['EBIT'].values[0] > dre.loc['EBIT'].values[1])
                variacao_caixa = int(dfc.loc['Changes In Cash'].values[0] > dfc.loc['Changes In Cash'].values[1])
                variacao_divida = int(balanco.loc['Current Liabilities'].values[0] < balanco.loc['Current Liabilities'].values[1])
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida
                print(soma_fundamentos)       
                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    list_relacao_mm.append(abs(hist_precos.iloc[-1] / np.mean(hist_precos.iloc[-10:])-1))
                    list_precos.append(round(hist_precos.iloc[-1],2))
            else:
                continue
        except:
            print('Erro - ' + ativo)
            continue

In [34]:
carteira_df = pd.DataFrame({'MM':list_relacao_mm,'Preços':list_precos}, index=list_melhores).sort_values(by = 'MM',ascending=True)

carteira_df

,MM,Preços
AURE3,0.005541,13.61
SLCE3,0.014539,37.96
CPFE3,0.021747,34.72
GGBR4,0.021803,22.73
PETR4,0.031855,34.10
BEEF3,0.032919,7.97
TAEE11,0.035472,35.73
CMIG4,0.037666,12.70
PETR3,0.040526,36.65
GOAU4,0.045696,11.03


In [35]:
lista_drop = ['PETR3']
carteira_limpa = carteira_df.drop(lista_drop)
carteira_limpa

,MM,Preços
AURE3,0.005541,13.61
SLCE3,0.014539,37.96
CPFE3,0.021747,34.72
GGBR4,0.021803,22.73
PETR4,0.031855,34.10
BEEF3,0.032919,7.97
TAEE11,0.035472,35.73
CMIG4,0.037666,12.70
GOAU4,0.045696,11.03
JBSS3,0.048419,21.48


In [37]:
list_pesos = [(1+x)/(carteira_limpa['MM'].sum()+len(carteira_limpa['MM'].values)) for x in carteira_limpa['MM'].values]
list_pesos.reverse()

carteira_limpa['Pesos'] = list_pesos
carteira_limpa

,MM,Preços,Pesos
AURE3,0.005541,13.61,0.081372
SLCE3,0.014539,37.96,0.079608
CPFE3,0.021747,34.72,0.079182
GGBR4,0.021803,22.73,0.077375
PETR4,0.031855,34.10,0.077174
BEEF3,0.032919,7.97,0.076582
TAEE11,0.035472,35.73,0.076420
CMIG4,0.037666,12.70,0.076231
GOAU4,0.045696,11.03,0.076153
JBSS3,0.048419,21.48,0.075411


In [38]:
patrimonio_inicial = 1200

carteira_limpa['Lote'] = round((carteira_limpa['Pesos'] * patrimonio_inicial/carteira_limpa['Preços'])/100,2)*100
carteira_limpa

,MM,Preços,Pesos,Lote
AURE3,0.005541,13.61,0.081372,7.0
SLCE3,0.014539,37.96,0.079608,3.0
CPFE3,0.021747,34.72,0.079182,3.0
GGBR4,0.021803,22.73,0.077375,4.0
PETR4,0.031855,34.10,0.077174,3.0
BEEF3,0.032919,7.97,0.076582,12.0
TAEE11,0.035472,35.73,0.076420,3.0
CMIG4,0.037666,12.70,0.076231,7.0
GOAU4,0.045696,11.03,0.076153,8.0
JBSS3,0.048419,21.48,0.075411,4.0


In [39]:
carteira_final = carteira_limpa[['Lote','Preços']]
carteira_final

,Lote,Preços
AURE3,7.0,13.61
SLCE3,3.0,37.96
CPFE3,3.0,34.72
GGBR4,4.0,22.73
PETR4,3.0,34.10
BEEF3,12.0,7.97
TAEE11,3.0,35.73
CMIG4,7.0,12.70
GOAU4,8.0,11.03
JBSS3,4.0,21.48
